In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time 
import random
import mmap
import pickle

#parameters
block_size = 64
batch_size = 128
max_iters = 600
eval_iters = 100
learning_rate = 3e-4
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2


In [5]:
chars = ""
with open('openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [6]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)
#print (data[:100])

In [7]:
#memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split == 'train' else "openwebtext/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access = mmap.ACCESS_READ) as mm:
            #Determine the file size and a radom position to start readin
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)
            #seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)
            #Decode the block to a string, ignoring any invalid bye sequences
            decode_block = block.decode('utf-8', errors = 'ignore').replace('\r','')

            #Traind and test split
            data = torch.tensor(encode(decode_block), dtype = torch.long)
    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y
    
#codes below used to train a small sample dataset  
# def get_batch     
# n = int(0.8*len(data))
# train_data = data[:n]
# val_data = data[n:]
# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x,y = x.to(device), y.to(device)
#     return x, y
# x, y = get_batch('train')  
# print ('inputs: ')
# print (x)
# print('targets: ')
# print (y)

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        #Avoid to reinitialise initial head, saving computational time. 
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #training without this will take longer)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        #input of size (batch, time-step, channels)
        #output of size (batch, time-step, head size)
        B, T, C = x.shape #(batch, time, channel)
        k = self.key(x)
        q = self.query(x)
        #compute attentin score "affinities"
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**0.5 #(B, T, hs) @ (B, hs, T) ->(B,T,T) -2 2nd last dim, -1 last dim, k.shape[-1]**0.5 is scaling
        wei = wei.masked_fill(self.tril[:T, :T] ==0, float('-inf')) #(B,T,T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) 
        v = self.value(x) #(B, T, hs)
        out = wei @ v #(B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
    
class MultiHeadAttention(nn.Module):
    """Multiheads of attention of self attention in parallel"""
   
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear (head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #last dimension is (batch B, Time T, Feature F)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""
   
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout) #prevent over fitting and other problems
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    # transformer block: communidation followedd by computation
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward (self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__ (self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights) #calculate stanard deviation

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        B, T = index.shape
       
        #index and targets are both (B,T)
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss =None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
         
        return logits, loss
  
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the prediction
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] #b,c
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(b,c)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(b,1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(b, t)
        return index
   
model = GPTLanguageModel(vocab_size)
#Loading model parameters
# with open ('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
#load successfully    
m = model.to(device)


In [ ]:
#create a pytorch optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):    
    print ("iter % eval_iters", iter % eval_iters, " ", time.strftime("%H:%M:%S", time.localtime()))
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss:{losses['train']:.3f}, val loss:{losses['val']:.3f}")
    xb, yb = get_batch('train')
    
    #evaluate the loss. This is the standard architecture to evaluate the loss.
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl','wb') as f:
    pickle.dump(model,f)
print ('model saved')
    
    

iter % eval_iters 0   16:39:40


In [1]:
#context = torch.zeros((1, 1), dtype = torch.long, device = device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)